# Heston Calibration

We calibrate the Heston model on real SPX options data and compare the resulting smile to BS.
Heston fits better — but we'll see where it breaks down.

In [ ]:
!git clone https://github.com/adnanegrb/rough-volatility-calibrator
import os
os.chdir('rough-volatility-calibrator')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
from datetime import datetime
from scipy.optimize import minimize
from models.black_scholes import implied_vol
from models.heston import heston_mc

In [ ]:
# Pull same SPX data as notebook 01
spx = yf.Ticker('^SPX')
S = spx.history(period='1d')['Close'].iloc[-1]

expiry = spx.options[4]
chain  = spx.option_chain(expiry)
calls  = chain.calls

calls = calls[(calls['strike'] > S * 0.85) & (calls['strike'] < S * 1.15)]
calls = calls[calls['lastPrice'] > 0.5]
calls = calls[calls['openInterest'] > 1000].reset_index(drop=True)

T = (datetime.strptime(expiry, '%Y-%m-%d') - datetime.today()).days / 365
r = 0.05

strikes      = calls['strike'].values
market_prices = calls['lastPrice'].values

# Market implied vols
market_ivols = np.array([implied_vol(p, S, K, T, r) for p, K in zip(market_prices, strikes)])
mask = ~np.isnan(market_ivols)
strikes, market_ivols = strikes[mask], market_ivols[mask]

In [ ]:
# Calibration — minimize RMSE between Heston smile and market smile
def objective(params):
    v0, kappa, theta, xi, rho = params
    if v0 <= 0 or kappa <= 0 or theta <= 0 or xi <= 0 or not (-1 < rho < 0):
        return 1e6
    heston_ivols = []
    for K in strikes:
        price = heston_mc(S, K, T, r, v0, kappa, theta, xi, rho, n_paths=10000, n_steps=100)
        iv = implied_vol(price, S, K, T, r)
        heston_ivols.append(iv)
    heston_ivols = np.array(heston_ivols)
    mask = ~np.isnan(heston_ivols)
    return np.sqrt(np.mean((heston_ivols[mask] - market_ivols[mask])**2))

x0     = [0.04, 2.0, 0.04, 0.3, -0.7]
bounds = [(1e-4, 0.5), (0.1, 10), (1e-4, 0.5), (0.01, 1.0), (-0.99, -0.01)]

print('Calibrating Heston... (this takes a few minutes)')
result = minimize(objective, x0, method='Nelder-Mead', bounds=bounds,
                  options={'maxiter': 100, 'xatol': 1e-3})

v0_cal, kappa_cal, theta_cal, xi_cal, rho_cal = result.x
print(f'v0={v0_cal:.4f}  kappa={kappa_cal:.4f}  theta={theta_cal:.4f}  xi={xi_cal:.4f}  rho={rho_cal:.4f}')

In [ ]:
# Heston smile with calibrated params
heston_ivols = []
for K in strikes:
    price = heston_mc(S, K, T, r, v0_cal, kappa_cal, theta_cal, xi_cal, rho_cal)
    iv = implied_vol(price, S, K, T, r)
    heston_ivols.append(iv)
heston_ivols = np.array(heston_ivols)

atm_vol = market_ivols[np.argmin(np.abs(strikes - S))]

fig, ax = plt.subplots(figsize=(10, 5))

ax.plot(strikes, market_ivols * 100, 'o-', color='steelblue', linewidth=2, label='Market')
ax.plot(strikes, heston_ivols * 100, 's--', color='darkorange', linewidth=2, label='Heston')
ax.axhline(y=atm_vol * 100, color='tomato', linestyle=':', linewidth=1.5, label='BS flat vol')

ax.set_xlabel('Strike')
ax.set_ylabel('Implied Volatility (%)')
ax.set_title(f'Heston vs Market  |  Expiry {expiry}  |  Spot {S:.0f}')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
os.makedirs('results', exist_ok=True)
plt.savefig('results/heston_smile.png', dpi=150)

from google.colab import files
files.download('results/heston_smile.png')

plt.show()

Heston fits the smile better than BS — but calibration on short maturities remains a challenge.
This is precisely where Rough Bergomi takes over.